In [ ]:
#conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
#!pip install tensorflow-gpu
#!pip install tensorflow_hub
#!pip install tqdm

#export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_PREFIX/lib/

In [ ]:
#source data_utils/gen_bbox.py

In [3]:
import os
import glob
import json
import queue
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from PIL import Image
from tqdm.notebook import tqdm

In [4]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [10]:
def hateful_meme(img_dir, output_path, debug=False):
    with tf.device('/device:GPU:0'):
        module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]
        detector = hub.load(module_handle).signatures['default']
        det_annos = []
        
        img_pattern = os.path.join(img_dir, '*.png')
        img_files = glob.glob(img_pattern)
        img_files = sorted(img_files)

        for img_file in tqdm(img_files):
            img_tensor = tf.io.read_file(img_file)
            img_tensor = tf.image.decode_image(img_tensor, channels=3)
            tensor = tf.image.convert_image_dtype(img_tensor, tf.float32)[tf.newaxis, ...]
            
            result = detector(tensor)
            result = {key:value.numpy() for key,value in result.items()}
            
            boxes_score = zip(
                            result["detection_boxes"],
                            result["detection_scores"],
                            result["detection_class_entities"],
                            result["detection_class_labels"])
            boxes_score = [
                {
                    'ymin': float(b[0]),
                    'xmin': float(b[1]),
                    'ymax': float(b[2]),
                    'xmax': float(b[3]),
                    'score': float(s),
                    'class_name': c.decode("ascii"),
                    'class_id': int(ci),
                }
                for b, s, c, ci in list(boxes_score) if s > 0.2]
            
            img_name = os.path.basename(img_file)
            det_anno = {
                'img_name': img_name,
                'boxes_and_score': boxes_score
            }
            det_annos.append(det_anno)
            
        with open(output_path, mode='w') as output:
            json.dump(det_annos, output)

In [6]:
image_name = '01235.png'
data_dir = '/media/alex/Storage/coding/HimariO/HatefulMemesChallenge/data/hateful_memes/'

In [11]:
hateful_meme(os.path.join(data_dir, 'img_clean'),
             os.path.join(data_dir, 'box_annos.json')
            )

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [12]:
#Пример работы кода
module_handle = "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1" #@param ["https://tfhub.dev/google/openimages_v4/ssd/mobilenet_v2/1", "https://tfhub.dev/google/faster_rcnn/openimages_v4/inception_resnet_v2/1"]
detector = hub.load(module_handle).signatures['default']


img_tensor = tf.io.read_file(os.path.join(data_dir, 'img', image_name))
img_tensor = tf.image.decode_image(img_tensor, channels=3)
converted_img = tf.image.convert_image_dtype(img_tensor, tf.float32)[tf.newaxis, ...]

result = detector(converted_img)
result = {key:value.numpy() for key,value in result.items()}


boxes_score = zip(
    result["detection_boxes"],
    result["detection_scores"],
    result["detection_class_entities"],
    result["detection_class_labels"])

boxes_score = [
    {
        'ymin': float(b[0]),
        'xmin': float(b[1]),
        'ymax': float(b[2]),
        'xmax': float(b[3]),
        'score': float(s),
        'class_name': c.decode("ascii"),
        'class_id': int(ci),
    }
    for b, s, c, ci in list(boxes_score) if s > 0.2]
        
boxes_score

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


[{'ymin': 0.26804885268211365,
  'xmin': 0.3950812816619873,
  'ymax': 0.6153359413146973,
  'xmax': 0.5982983112335205,
  'score': 0.9718632102012634,
  'class_name': 'Human face',
  'class_id': 502},
 {'ymin': 0.14357806742191315,
  'xmin': 0.04331391677260399,
  'ymax': 0.9951313734054565,
  'xmax': 0.9685384035110474,
  'score': 0.9124561548233032,
  'class_name': 'Man',
  'class_id': 308},
 {'ymin': 0.4765247106552124,
  'xmin': 0.07855606824159622,
  'ymax': 0.980301558971405,
  'xmax': 0.9747498631477356,
  'score': 0.6230961084365845,
  'class_name': 'Clothing',
  'class_id': 433},
 {'ymin': 0.1488388329744339,
  'xmin': 0.36896005272865295,
  'ymax': 0.3470507562160492,
  'xmax': 0.6266028881072998,
  'score': 0.32622191309928894,
  'class_name': 'Hat',
  'class_id': 161},
 {'ymin': 0.46257153153419495,
  'xmin': 0.02542208880186081,
  'ymax': 0.9678794741630554,
  'xmax': 0.3584445118904114,
  'score': 0.23643866181373596,
  'class_name': 'Human arm',
  'class_id': 503}]